생산적대 신경망: 합성하여 가상으로 만들어냄

https://www.thispersondoesnotexist.com/

이안 굿펠러 논문 내용 :
간 변경 알고리즘 위조 지폐
- 위조 지폐범은 가짜 지폐를  만들기 위해 노력하고, 경찰은 가짜를 찾기 위해 노력한다 
- 한쪽은 가짜를 만들고, 한쪽은 진짜와 비교하는 경험
- 가짜를 만들어내는 것은 '생성자(Generator)', 진위를 가려내는 파트를 '판별자(Discriminator)' 라고 부른다 

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
if not os.path.exists('./gan_images'):
    os.makedirs('./gan_images')

In [16]:
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
# 들어오는 값에 0.2를 사용
generator.add(BatchNormalization())
# 가중치 값들을 평균이 0, 가중치 값을 1로 만들어 주는 함수, 가중치가 밖으로 튀지 않도록 잡아준다
generator.add(Reshape((7,7,128))) #1차원으로 나온 128*7*7 개를 3차원으로 재편성
generator.add(UpSampling2D())
#Upsampling2D : Maxpooling2D는 이미지 편접 시 값이 줄어들었던 것을 다시 확장하며 출력값과 입력값이 동일하게 만들어주는 것
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh')) 
# 실제 이미지를 만들기 위해서 출력은 하나가 나오지만 

In [17]:
generator.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              633472    
_________________________________________________________________
batch_normalization_2 (Batch (None, 6272)              25088     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 64)       

In [18]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding = 'same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

In [19]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False
# discriminator 은 디포릍 값으로 학습을 하지 않는 값을 준다. 
# generator 학습 시, discriminator 은 학습을 하면 안된다. 진위와 판별해야 하기 때문에, 팔변할 떄마다 기준이 바뀌면 곤란하기 때문에 

In [20]:
discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_5 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_6 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)             

In [21]:
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
# shape = (100) 100 차원을 넣고, 그것을 generator 가 받는다. 
# generator 을 discrimiator 가 맞고, 그것을 dis_ouput 이 받는다. 
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [30]:
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float')
    X_train = (X_train - 127.5) / 127.5
    
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for i in range(epoch):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)
        
        print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)
        
        if i % saving_interval == 0:
            noise = np.random.normal(0, 1, (25, 100))
            gen_imgs = generator.predict(noise)
            
            gen_imgs = 0.5 * gen_imgs + 0.5 # 0 ~ 1 rescale
            
            fig, axs = plt.subplots(5, 5)
            count = 0
            for j in range(5):
                for k in range(5):
                    axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                    axs[j, k].axis('off')
                    count += 1
            fig.savefig('gan_images/gan_mnist_%d.png' % i)

In [31]:
gan_train(4001, 32,200)

epoch:0  d_loss:0.5636  g_loss:0.3678
epoch:1  d_loss:0.6391  g_loss:0.1626
epoch:2  d_loss:0.5367  g_loss:0.1593
epoch:3  d_loss:0.4594  g_loss:0.2331
epoch:4  d_loss:0.4492  g_loss:0.3380
epoch:5  d_loss:0.4354  g_loss:0.4263
epoch:6  d_loss:0.4653  g_loss:0.3852
epoch:7  d_loss:0.4824  g_loss:0.3168
epoch:8  d_loss:0.4845  g_loss:0.2664
epoch:9  d_loss:0.4791  g_loss:0.2341
epoch:10  d_loss:0.4713  g_loss:0.2464
epoch:11  d_loss:0.4216  g_loss:0.3525
epoch:12  d_loss:0.3857  g_loss:0.5346
epoch:13  d_loss:0.3768  g_loss:0.6679
epoch:14  d_loss:0.3466  g_loss:0.6928
epoch:15  d_loss:0.3092  g_loss:0.7736
epoch:16  d_loss:0.2785  g_loss:0.9364
epoch:17  d_loss:0.3248  g_loss:1.0006
epoch:18  d_loss:0.3141  g_loss:1.1286
epoch:19  d_loss:0.3134  g_loss:1.0567
epoch:20  d_loss:0.3991  g_loss:0.6777
epoch:21  d_loss:0.2932  g_loss:0.6269
epoch:22  d_loss:0.2235  g_loss:0.6740
epoch:23  d_loss:0.2124  g_loss:0.8490
epoch:24  d_loss:0.2740  g_loss:0.2748
epoch:25  d_loss:0.2426  g_loss:0.5

/Users/jeongsarah/opt/anaconda3/envs/python-env/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


epoch:3801  d_loss:0.4500  g_loss:1.3062
epoch:3802  d_loss:0.4628  g_loss:1.4539
epoch:3803  d_loss:0.5351  g_loss:1.8429
epoch:3804  d_loss:0.5198  g_loss:1.8678
epoch:3805  d_loss:0.6099  g_loss:2.0097
epoch:3806  d_loss:0.7098  g_loss:1.4865
epoch:3807  d_loss:0.4756  g_loss:1.4446
epoch:3808  d_loss:0.6402  g_loss:1.6123
epoch:3809  d_loss:0.5563  g_loss:1.3279
epoch:3810  d_loss:0.4963  g_loss:1.8450
epoch:3811  d_loss:0.5558  g_loss:1.7994
epoch:3812  d_loss:0.4349  g_loss:2.0034
epoch:3813  d_loss:0.4623  g_loss:1.8801
epoch:3814  d_loss:0.5693  g_loss:1.9066
epoch:3815  d_loss:0.4397  g_loss:1.6523
epoch:3816  d_loss:0.4710  g_loss:1.4391
epoch:3817  d_loss:0.4710  g_loss:1.3894
epoch:3818  d_loss:0.5756  g_loss:1.1793
epoch:3819  d_loss:0.5018  g_loss:1.5688
epoch:3820  d_loss:0.4203  g_loss:2.0671
epoch:3821  d_loss:0.5185  g_loss:1.7692
epoch:3822  d_loss:0.5000  g_loss:1.6455
epoch:3823  d_loss:0.4512  g_loss:1.8190
epoch:3824  d_loss:0.4685  g_loss:1.5160
epoch:3825  d_lo

/Users/jeongsarah/opt/anaconda3/envs/python-env/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [2]:
# reaky_ReLU 설명
import numpy as np

def leaky_relu_function(x):
    return np.where(x <= 0, 0.01*x, x)

x = np.linspace(-5,5)
y = leaky_relu_function(x)

plt(x,y)
plt.show()

NameError: name 'plt' is not defined